# Import

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.1/547.1 kB 12.9 MB/s eta 0:00:00


In [3]:
import shap
import pandas as pd
import tensorflow as tf
from tensorflow import keras

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


# Load trainset

In [4]:
df = pd.read_csv('/content/drive/MyDrive/Dataset/insdn/binary/train.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275111 entries, 0 to 275110
Data columns (total 77 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Flow Duration      275111 non-null  float64
 1   Tot Fwd Pkts       275111 non-null  float64
 2   Tot Bwd Pkts       275111 non-null  float64
 3   TotLen Fwd Pkts    275111 non-null  float64
 4   TotLen Bwd Pkts    275111 non-null  float64
 5   Fwd Pkt Len Max    275111 non-null  float64
 6   Fwd Pkt Len Min    275111 non-null  float64
 7   Fwd Pkt Len Mean   275111 non-null  float64
 8   Fwd Pkt Len Std    275111 non-null  float64
 9   Bwd Pkt Len Max    275111 non-null  float64
 10  Bwd Pkt Len Min    275111 non-null  float64
 11  Bwd Pkt Len Mean   275111 non-null  float64
 12  Bwd Pkt Len Std    275111 non-null  float64
 13  Flow Byts/s        275111 non-null  float64
 14  Flow Pkts/s        275111 non-null  float64
 15  Flow IAT Mean      275111 non-null  float64
 16  Fl

In [5]:
df = df.drop(columns=['Label'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275111 entries, 0 to 275110
Data columns (total 76 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Flow Duration      275111 non-null  float64
 1   Tot Fwd Pkts       275111 non-null  float64
 2   Tot Bwd Pkts       275111 non-null  float64
 3   TotLen Fwd Pkts    275111 non-null  float64
 4   TotLen Bwd Pkts    275111 non-null  float64
 5   Fwd Pkt Len Max    275111 non-null  float64
 6   Fwd Pkt Len Min    275111 non-null  float64
 7   Fwd Pkt Len Mean   275111 non-null  float64
 8   Fwd Pkt Len Std    275111 non-null  float64
 9   Bwd Pkt Len Max    275111 non-null  float64
 10  Bwd Pkt Len Min    275111 non-null  float64
 11  Bwd Pkt Len Mean   275111 non-null  float64
 12  Bwd Pkt Len Std    275111 non-null  float64
 13  Flow Byts/s        275111 non-null  float64
 14  Flow Pkts/s        275111 non-null  float64
 15  Flow IAT Mean      275111 non-null  float64
 16  Fl

# Load adv samples

In [6]:
feature_list = ['Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts',
       'TotLen Bwd Pkts', 'Fwd Pkt Len Max', 'Fwd Pkt Len Min',
       'Fwd Pkt Len Mean', 'Fwd Pkt Len Std', 'Bwd Pkt Len Max',
       'Bwd Pkt Len Min', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Flow Byts/s',
       'Flow Pkts/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Tot', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Len',
       'Bwd Header Len', 'Fwd Pkts/s', 'Bwd Pkts/s', 'Pkt Len Min',
       'Pkt Len Max', 'Pkt Len Mean', 'Pkt Len Std', 'Pkt Len Var',
       'FIN Flag Cnt', 'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt',
       'ACK Flag Cnt', 'URG Flag Cnt', 'CWE Flag Count', 'ECE Flag Cnt',
       'Down/Up Ratio', 'Pkt Size Avg', 'Fwd Seg Size Avg', 'Bwd Seg Size Avg',
       'Fwd Byts/b Avg', 'Fwd Pkts/b Avg', 'Fwd Blk Rate Avg',
       'Bwd Byts/b Avg', 'Bwd Pkts/b Avg', 'Bwd Blk Rate Avg',
       'Subflow Fwd Pkts', 'Subflow Fwd Byts', 'Subflow Bwd Pkts',
       'Subflow Bwd Byts', 'Init Fwd Win Byts', 'Init Bwd Win Byts',
       'Fwd Act Data Pkts', 'Fwd Seg Size Min', 'Active Mean', 'Active Std',
       'Active Max', 'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max',
       'Idle Min']

In [7]:
hsj_path = '/content/drive/MyDrive/Dataset/insdn/adv/24_feature/hsj/hsj_bin_feature'

In [8]:
hsj_df = pd.read_csv(hsj_path)

In [9]:
#hsj_df.columns = feature_list
hsj_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55151 entries, 0 to 55150
Data columns (total 76 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Flow Duration      55151 non-null  float64
 1   Tot Fwd Pkts       55151 non-null  float64
 2   Tot Bwd Pkts       55151 non-null  float64
 3   TotLen Fwd Pkts    55151 non-null  float64
 4   TotLen Bwd Pkts    55151 non-null  float64
 5   Fwd Pkt Len Max    55151 non-null  float64
 6   Fwd Pkt Len Min    55151 non-null  float64
 7   Fwd Pkt Len Mean   55151 non-null  float64
 8   Fwd Pkt Len Std    55151 non-null  float64
 9   Bwd Pkt Len Max    55151 non-null  float64
 10  Bwd Pkt Len Min    55151 non-null  float64
 11  Bwd Pkt Len Mean   55151 non-null  float64
 12  Bwd Pkt Len Std    55151 non-null  float64
 13  Flow Byts/s        55151 non-null  float64
 14  Flow Pkts/s        55151 non-null  float64
 15  Flow IAT Mean      55151 non-null  float64
 16  Flow IAT Std       551

# Load DL-based IDS model

In [10]:
model_mlp = tf.keras.models.load_model('/content/drive/MyDrive/Dataset/insdn/insdn_mlp_bin.h5', compile=True)

# SHAP

In [11]:
explainer = shap.KernelExplainer(model_mlp, shap.sample(df, 100, random_state = 42))

In [12]:
hsj_1000 = hsj_df.head(1000)

In [13]:
shap_values = explainer.shap_values(hsj_1000)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [14]:
print(shap_values[0])

[[ 0.01241914  0.00067478  0.00230754 ... -0.00230382  0.00363369
   0.00222544]
 [ 0.          0.00200102 -0.00139842 ... -0.00281277  0.00416309
   0.00723277]
 [ 0.0016717  -0.001468    0.         ... -0.00120473  0.00401349
   0.00616336]
 ...
 [ 0.          0.00766265  0.         ...  0.          0.00963323
   0.00912605]
 [ 0.01315727  0.00214782 -0.00225441 ...  0.          0.00320638
   0.00330733]
 [ 0.00569689 -0.00263478 -0.00418095 ...  0.0047994   0.
   0.        ]]


In [15]:
len(shap_values[0])

1000

In [16]:
type(shap_values[0])

numpy.ndarray

In [17]:
data_df = pd.DataFrame(shap_values[0])

In [18]:
data_df.head()

,0,1,2,3,4,5,6,7,8,9,...,66,67,68,69,70,71,72,73,74,75
0,0.012419,0.000675,0.002308,0.000000,0.000000,0.004645,0.001602,0.004087,0.004573,-0.001706,...,0.000000,0.0,0.000000,0.000000,0.000000,-0.001494,0.002422,-0.002304,0.003634,0.002225
1,0.000000,0.002001,-0.001398,0.000000,0.000000,-0.005961,-0.001486,-0.003461,-0.008009,0.002688,...,-0.001194,0.0,0.000000,0.001071,-0.001793,-0.001264,0.005498,-0.002813,0.004163,0.007233
2,0.001672,-0.001468,0.000000,0.000930,-0.000634,0.001029,0.000000,0.000331,0.002229,0.001307,...,0.000000,0.0,-0.001061,0.000000,0.000394,-0.000916,0.001920,-0.001205,0.004013,0.006163
3,0.000395,0.000639,0.000000,0.001228,0.000000,0.002199,0.000000,0.002428,0.001229,0.000789,...,0.000000,0.0,-0.001513,-0.000564,-0.000194,0.000000,0.002652,0.000000,0.004844,0.005502
4,-0.037205,-0.004587,0.000000,0.000000,0.009399,0.000000,0.000000,0.005916,0.000000,0.016184,...,-0.006656,0.0,0.000000,0.006625,0.000000,0.000000,-0.042873,0.000000,-0.058399,-0.055738


In [ ]:
data_df.to_csv('/content/drive/MyDrive/DACN_XAI_Adv_defense/InSDN/MLP_HSJ/1000_samples_[0]_bin.csv', index=False)

In [19]:
data_df.columns = feature_list
data_df.head()

,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,...,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,0.012419,0.000675,0.002308,0.000000,0.000000,0.004645,0.001602,0.004087,0.004573,-0.001706,...,0.000000,0.0,0.000000,0.000000,0.000000,-0.001494,0.002422,-0.002304,0.003634,0.002225
1,0.000000,0.002001,-0.001398,0.000000,0.000000,-0.005961,-0.001486,-0.003461,-0.008009,0.002688,...,-0.001194,0.0,0.000000,0.001071,-0.001793,-0.001264,0.005498,-0.002813,0.004163,0.007233
2,0.001672,-0.001468,0.000000,0.000930,-0.000634,0.001029,0.000000,0.000331,0.002229,0.001307,...,0.000000,0.0,-0.001061,0.000000,0.000394,-0.000916,0.001920,-0.001205,0.004013,0.006163
3,0.000395,0.000639,0.000000,0.001228,0.000000,0.002199,0.000000,0.002428,0.001229,0.000789,...,0.000000,0.0,-0.001513,-0.000564,-0.000194,0.000000,0.002652,0.000000,0.004844,0.005502
4,-0.037205,-0.004587,0.000000,0.000000,0.009399,0.000000,0.000000,0.005916,0.000000,0.016184,...,-0.006656,0.0,0.000000,0.006625,0.000000,0.000000,-0.042873,0.000000,-0.058399,-0.055738


In [20]:
data_dict = {}
for idx in range(0, 1000):
  # Select the desired row and get the top 10 values's column names
  row_values = data_df.iloc[idx].nlargest(10).index.tolist()

  # Create a dictionary and add the list with filename as the key
  data_dict['Adv sample ' + str(idx)] = row_values

In [21]:
print(data_dict)

{'Adv sample 0': ['Init Bwd Win Byts', 'Down/Up Ratio', 'Pkt Len Min', 'Flow Duration', 'Fwd IAT Max', 'Flow IAT Std', 'Pkt Len Max', 'Pkt Len Std', 'Flow IAT Max', 'Pkt Size Avg'], 'Adv sample 1': ['SYN Flag Cnt', 'Init Bwd Win Byts', 'Idle Min', 'Idle Mean', 'Idle Max', 'Flow Pkts/s', 'Bwd Pkt Len Std', 'Pkt Len Min', 'Fwd IAT Max', 'Bwd Pkt Len Max'], 'Adv sample 2': ['Init Bwd Win Byts', 'SYN Flag Cnt', 'Pkt Len Min', 'Idle Min', 'Flow IAT Max', 'Idle Max', 'Bwd Pkt Len Min', 'Pkt Size Avg', 'Pkt Len Std', 'Pkt Len Max'], 'Adv sample 3': ['Init Bwd Win Byts', 'SYN Flag Cnt', 'Pkt Len Min', 'Idle Min', 'Idle Max', 'Pkt Len Max', 'Flow IAT Max', 'Bwd Pkt Len Min', 'Fwd IAT Max', 'Idle Mean'], 'Adv sample 4': ['ACK Flag Cnt', 'Bwd IAT Std', 'Init Bwd Win Byts', 'Bwd IAT Tot', 'Bwd IAT Max', 'Bwd Pkt Len Max', 'Flow IAT Mean', 'TotLen Bwd Pkts', 'Pkt Len Var', 'Pkt Len Min'], 'Adv sample 5': ['Bwd Pkts/s', 'Init Bwd Win Byts', 'Down/Up Ratio', 'Pkt Len Min', 'Flow Duration', 'Fwd IAT M

In [22]:
len(shap_values[0])

1000

In [23]:
len(data_dict)

1000

# Detection phase

In [24]:
import os
import pandas as pd

In [30]:
whitelist_df = pd.read_csv('/content/drive/MyDrive/MLP_top24_InSDN.csv')
whitelist_df.head(24)

,Feature,Frequency
0,Init Bwd Win Byts,786
1,Idle Max,546
2,Bwd Pkt Len Max,531
3,Idle Mean,449
4,Tot Fwd Pkts,432
5,Idle Min,413
6,Bwd Pkt Len Std,399
7,Tot Bwd Pkts,395
8,Bwd Pkt Len Mean,382
9,TotLen Fwd Pkts,373


## Detection rate (top 51) = 956/1000

In [37]:
whitelist = whitelist_df.iloc[:, 0][:51].tolist()
print(whitelist)

['Init Bwd Win Byts', 'Idle Max', 'Bwd Pkt Len Max', 'Idle Mean', 'Tot Fwd Pkts', 'Idle Min', 'Bwd Pkt Len Std', 'Tot Bwd Pkts', 'Bwd Pkt Len Mean', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Bwd IAT Max', 'Bwd IAT Tot', 'Fwd IAT Max', 'Bwd Seg Size Avg', 'Pkt Len Min', 'SYN Flag Cnt', 'Flow Duration', 'Fwd Pkt Len Std', 'Fwd IAT Tot', 'Bwd IAT Std', 'Flow Pkts/s', 'Down/Up Ratio', 'Fwd IAT Std', 'Fwd Pkt Len Min', 'Fwd Pkt Len Max', 'Subflow Bwd Pkts', 'Bwd Pkt Len Min', 'Flow IAT Mean', 'Flow IAT Min', 'Fwd IAT Min', 'Bwd IAT Mean', 'Bwd IAT Min', 'Bwd Header Len', 'Subflow Bwd Byts', 'Subflow Fwd Byts', 'Fwd Header Len', 'Bwd PSH Flags', 'Idle Std', 'Fwd Pkts/s', 'Fwd Act Data Pkts', 'Pkt Len Max', 'Fwd IAT Mean', 'Active Max', 'Active Std', 'Pkt Len Std', 'Pkt Len Var', 'Subflow Fwd Pkts', 'Flow Byts/s', 'Pkt Size Avg', 'Active Mean']


In [38]:
alert = []

# Iterate over each key-value pair in the data_dict
for key, value in data_dict.items():
    # Check if there are one feature is not in the whitelist
    for string in value:
      if string not in whitelist:
        alert.append(key)
        break

# Print the alert dictionary
print("Alert list length:" + str(len(alert)))

Alert list length:956


## Detection rate (top 59) = 345/1000



In [33]:
whitelist2 = whitelist_df.iloc[:, 0][:59].tolist()
print(whitelist2)

['Init Bwd Win Byts', 'Idle Max', 'Bwd Pkt Len Max', 'Idle Mean', 'Tot Fwd Pkts', 'Idle Min', 'Bwd Pkt Len Std', 'Tot Bwd Pkts', 'Bwd Pkt Len Mean', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Bwd IAT Max', 'Bwd IAT Tot', 'Fwd IAT Max', 'Bwd Seg Size Avg', 'Pkt Len Min', 'SYN Flag Cnt', 'Flow Duration', 'Fwd Pkt Len Std', 'Fwd IAT Tot', 'Bwd IAT Std', 'Flow Pkts/s', 'Down/Up Ratio', 'Fwd IAT Std', 'Fwd Pkt Len Min', 'Fwd Pkt Len Max', 'Subflow Bwd Pkts', 'Bwd Pkt Len Min', 'Flow IAT Mean', 'Flow IAT Min', 'Fwd IAT Min', 'Bwd IAT Mean', 'Bwd IAT Min', 'Bwd Header Len', 'Subflow Bwd Byts', 'Subflow Fwd Byts', 'Fwd Header Len', 'Bwd PSH Flags', 'Idle Std', 'Fwd Pkts/s', 'Fwd Act Data Pkts', 'Pkt Len Max', 'Fwd IAT Mean', 'Active Max', 'Active Std', 'Pkt Len Std', 'Pkt Len Var', 'Subflow Fwd Pkts', 'Flow Byts/s', 'Pkt Size Avg', 'Active Mean', 'ACK Flag Cnt', 'Flow IAT Max', 'Active Min', 'Pkt Len Mean', 'Fwd Pkt Len Mean', 'Flow IAT Std', 'Fwd Seg Size Avg', 'PSH Flag Cnt']


In [34]:
alert2 = []

# Iterate over each key-value pair in the data_dict
for key, value in data_dict.items():
    # Check if there are one feature is not in the whitelist
    for string in value:
      if string not in whitelist2:
        alert2.append(key)
        break

# Print the alert dictionary
print("Alert list length:" + str(len(alert2)))

Alert list length:345
